# Introduction

`Fit-DCS` is a Python toolbox for modeling and analyzing diffuse correlation spectroscopy (DCS) data. It provides tools for both forward modeling
and inverse fitting of DCS autocorrelation functions, allowing users to extract information about scatterer dynamics in turbid media.

This Jupyter Notebook provides an introduction to the main features of the `Fit-DCS` toolbox, complete with code examples and exercises to familiarize
users with its capabilities.

# Forward modeling

The simplest use of this toolbox is the modeling of the forward problem, that is, given the information on the scatterer dynamics
(Brownian diffusion coefficient `db` and/or mean square velocity `v_ms` depending on the scatterer model) and the geometry (e.g., semi-infinite,
slab, ...) calculate the corresponding autocorrelation functions `g1` and `g2`.

The `forward` module contains the expressions for the normalized and unnormalized first-order autocorrelations `g1_norm` and `g1`
obtained in the framework of the diffusion approximation to the correlation transport equation.
It contains one submodule per geometry: for example, `forward.homogeneous_inf` contains the solution for the homogeneous
semi-infinite geometry, `forward.homogeneous_slab` the ones for the homogeneous laterally infinite slab, and so on.

In addition to a geometric model, to simulate the autocorrelation functions we also need a model for scatterer motion.
These can be found in `forward.common`, and include Brownian motion, ballistic motion and hybrid motion with both components.

## Semi-infinite geometry

We'll start with a simple example of forward modeling in a semi-infinite geometry with Brownian motion.
First of all, let's import the necessary modules and define the parameters of the simulation.

In [ ]:
import numpy as np                                          # Arrays
import matplotlib.pyplot as plt                             # Plotting

from fit_dcs.forward import common                          # Common functions (e.g., scatterer motion models)
from fit_dcs.forward import homogeneous_semi_inf as hsi     # Semi-infinite geometry

tau = np.logspace(-8, 0, 200)               # Delay times, s
lambda0 = 785                               # Wavelength, nm
mua = 0.1                                   # Absorption coefficient, 1/cm
musp = 10                                   # Reduced scattering coefficient, 1/cm
rho = 2.5                                   # Source-detector separation, cm
n = 1.4                                     # Refractive index of the medium (external is 1)
db = 1e-8                                   # Brownian diffusion coefficient, cm^2/s
beta = 0.5                                  # Coherence of detected light

Note that, in order to be flexible, the geometric models in `Fit-DCS` do not take directly the Brownian diffusion coefficient `db` as input,
but rather the mean-square displacement (MSD) as a function of delay time `tau`.
This allows the user to define custom scatterer motion models if needed.

For simple Brownian motion, we can use the helper function `msd_brownian` in the `common` module to compute the MSD.

In [ ]:
msd = common.msd_brownian(tau, db)
g1_norm = hsi.g1_norm(msd, mua, musp, rho, n, lambda0)

We can now compute the second-order autocorrelation function `g2` using the Siegert relation and plot the resulting curves.

In [ ]:
g2_norm = 1 + beta * g1_norm**2

def plot_g1_g2(tau, g1, g2, labels=None):
    """
    Helper function to plot g1 and g2. Supports multiple curves with labels.
    """
    fig, axs = plt.subplots(2, 1, sharex=True)
    axs[0].semilogx(tau, g1.T, label=labels)
    axs[0].set_xlabel(r'$\tau$ (s)')
    axs[0].set_ylabel(r'$g_1$')
    if labels is not None:
        axs[0].legend()

    axs[1].semilogx(tau, g2.T, label=labels)
    axs[1].set_xlabel(r'$\tau$ (s)')
    axs[1].set_ylabel(r'$g_2$')
    if labels is not None:
        axs[1].legend()
    plt.tight_layout()
    plt.show()

plot_g1_g2(tau, g1_norm, g2_norm)

Let's now try different values of the Brownian diffusion coefficient `db` and see how they affect the autocorrelation functions.
We will define `g1` as a 2D array whose axes are `(db, tau)`, and loop over different `db` values to compute `g1` for each.
This 2D shape for `g1` and `g2` is what is expected by data analysis classes, where the first axis corresponds to
different iterations and the second axis to delay times.

In [ ]:
db_vals = [1e-9, 2e-9, 5e-9, 1e-8, 2e-8]
g1_all = np.zeros((len(db_vals), len(tau)))
for i in range(len(db_vals)):
    msd = common.msd_brownian(tau, db_vals[i])
    g1_all[i, :] = hsi.g1_norm(msd, mua, musp, rho, n, lambda0)
g2_all = 1 + beta * g1_all**2

plot_g1_g2(tau, g1_all, g2_all, labels=db_vals)  # We can pass the db values to be used in the legend

Now try writing some code to simulate `g1` and `g2` for the same geometry but with ballistic scatterer motion.
Use the function `common.msd_ballistic` to compute the mean-square displacement for ballistic motion.
Hover over the function name to see its documentation, including the input parameters.

In [ ]:
v_ms_vals = []  # Your code here... (mean square velocities, (cm/s)^2)

# Replace the zeros initialization with your code...
g1_all = np.zeros((len(v_ms_vals), len(tau)))
g2_all = np.zeros((len(v_ms_vals), len(tau)))

plot_g1_g2(tau, g1_all, g2_all, labels=v_ms_vals)

Finally, also try simulating `g1` and `g2` for a hybrid motion model with both Brownian and ballistic components.
You can use the function `common.msd_hybrid` to compute the mean-square displacement for hybrid motion.

In [ ]:
db_vals = []  # Your code here...
v_ms = 1e-4

# Replace the zeros initialization with your code...
g1_all = np.zeros((len(db_vals), len(tau)))
g2_all = np.zeros((len(db_vals), len(tau)))

plot_g1_g2(tau, g1_all, g2_all, labels=db_vals)

## Slab geometry

The procedure for forward modeling in a slab geometry is similar to the semi-infinite case.
We just need to import the appropriate module (`forward.homogeneous_inf_slab`) and define the additional
parameters for the slab geometry.
Since the theoretical model requires a sum over multiple image sources, we also need to define the number
of image sources to include in the calculation (`m_max`).

Note that two solutions are available for the slab geometry, one in reflectance (`g1_reflectance_norm`)
and one in transmittance (`g1_transmittance_norm`).

Let's see an example for the reflectance configuration.

In [ ]:
from fit_dcs.forward import homogeneous_inf_slab as slab

d = 0.5                                   # Slab thickness, cm
m_max = 10                                # Number of image sources to include
rho = 2.5                                 # Lateral source-detector separation, cm
db = 1e-8

msd = common.msd_brownian(tau, db)
g1_norm = slab.g1_reflectance_norm(msd, mua, musp, rho, n, lambda0, d, m_max)

g2_norm = 1 + beta * g1_norm**2
plot_g1_g2(tau, g1_norm, g2_norm)

Now try to verify that the slab solution in reflectance is the same as the semi-infinite solution
when we only consider the first image source (i.e., set `m_max = 0`).

In [ ]:
# Replace the zeros initialization with your code...
g1_semi_inf = np.zeros_like(tau)
g1_slab_m0 = np.zeros_like(tau)

# Plotting
plt.semilogx(tau, g1_semi_inf, label='Semi-infinite')
plt.semilogx(tau, g1_slab_m0, '--', label='Slab m=0')
plt.xlabel(r'$\tau$ (s)')
plt.ylabel(r'$g_1$')
plt.legend()
plt.show()

Finally, let's study the convergence of the slab solution as we increase the number of image sources `m_max`.
Try simulating `g1` and `g2` in the transmittance geometry for different values of `m_max`.

In [ ]:
mua = 0.05
musp = 10
d = 0.5
rho = 2  # Lateral source-detector separation, in transmittance it's usually 0 but here we want to see the effect of rho on convergence
db = 1e-8
msd = common.msd_brownian(tau, db)
m_max_vals = []  # Your code here...

# Replace the zeros initialization with your code...
g1_all = np.zeros((len(m_max_vals), len(tau)))
g2_all = np.zeros((len(m_max_vals), len(tau)))

plot_g1_g2(tau, g1_all, g2_all, labels=m_max_vals)

The parameters defined above were chosen to highlight the effect of increasing `m_max`. Try changing them
to see how they affect the convergence behavior. You should observe that higher absorption coefficients, thicker slabs,
and shorter source-detector separations require fewer image sources for convergence.

## Two-layer geometry

Finally, let's simulate the autocorrelation for a two-layer geometry.
The appropriate `g1_norm` function can be found in the `forward.bilayer` module.

The main difference with respect to the previous cases is that now we are dealing with two different media,
each with its scatterer motion. As such, in `forward.bilayer.g1_norm`, the `msd` argument is a 2D array of shape
`(2, len(tau))`, such that `msd[0, :]` is the mean-square displacement for the upper layer's scatterers and
`msd[1, :]` is the one relative to the lower layer. Of course, the scatterer motion models can be different in the two layers.

Additionally, the analytical expression of the two-layer geometry contains an integration over all (positive) spatial frequencies,
which in the implementation is truncated to the interval `(0, q_max)`, where `q_max` is an input parameter. As such, `q_max` should be
chosen sufficiently high to ensure convergence of the solution.

Let's see an example. For simplicity, we'll consider Brownian motion in both layers.

In [ ]:
import fit_dcs.forward.bilayer as bilayer

db_up = 1e-8
db_dn = 6e-8
msd_up = common.msd_brownian(tau, db_up)
msd_dn = common.msd_brownian(tau, db_dn)
msd = np.vstack((msd_up, msd_dn))  # Shape (2, len(tau))
mua_up = 0.1
mua_dn = 0.2
musp_up = 8
musp_dn = 10
n = 1.4
d = 1  # Thickness of the upper layer, cm
rho = 3
q_max = 80  # 1/cm

g1_norm = bilayer.g1_norm(msd, mua_up, mua_dn, musp_up, musp_dn, n, d, rho, lambda0, q_max)
g2_norm = 1 + beta * g1_norm**2

plot_g1_g2(tau, g1_norm, g2_norm)

To study the convergence of the solution, try simulating `g1` and `g2` for different values of `q_max`.

In [ ]:
q_max_vals = []  # Your code here...

# Replace the zeros initialization with your code...
g1_all = np.zeros((len(q_max_vals), len(tau)))
g2_all = np.zeros((len(q_max_vals), len(tau)))

plot_g1_g2(tau, g1_all, g2_all, labels=q_max_vals)

Now try investigating the effect of the lower layer's Brownian diffusion coefficient `db_dn` on the autocorrelation functions.
Generate `g1` and `g2` for different values of `db_dn`, while keeping `db_up` fixed.
You should use a value of `q_max` that ensures convergence, which you can determine from the previous exercise.

In [ ]:
db_dn_vals = []  # Your code here...

# Replace the zeros initialization with your code...
g1_all = np.zeros((len(db_dn_vals), len(tau)))
g2_all = np.zeros((len(db_dn_vals), len(tau)))

plot_g1_g2(tau, g1_all, g2_all, labels=db_dn_vals)

Finally, repeat the previous exercise with a different source-detector separation `rho`.
You should observe that the sensitivity to the lower layer's dynamics increases with `rho`.

You are also free to change other parameters (e.g., upper layer thickness `d`, scatterer motion model, ...) to see how they affect the results.

## Adding noise

So far all the curves we generated were noiseless, as they were obtained using the analytical solutions to the correlation diffusion equation.
In this section, we will see how to add realistic noise to the simulated `g2` curves.

The `utils.noise` module offers noise-related functionalities, most importantly the expression for the standard deviation of `g2_norm` as a
function of time delay `tau`.
This model was obtained for an exponentially decaying `g2_norm`, that is, `g2_norm = 1 + beta * exp(-tau/tau_c)`, which works best at short
time delays.

Let's plot the standard deviation. To get more realistic results, we will load the `tau` which comes from a hardware correlator,
rather than creating it ourselves as we did earlier. Additionally, the standard deviation depends on integration time, measurement countrate,
decay time constant, and number of detected speckles, so we will need to define those as well.

In [ ]:
from fit_dcs.utils import noise

tau = np.load("data/tau.npz")["tau_hardware"]

t_integration = 1   # Integration time, s
countrate = 70_000  # Measurement countrate, Hz
tau_c = 1e-5        # Exponential decay time constant, s
n_speckle = 1       # Number of speckles detected in parallel
sigma = noise.sigma_g2_norm(tau, t_integration, countrate, beta, tau_c, n_speckle)

# Plotting
plt.semilogx(tau, sigma)
plt.xlabel(r"$\tau$ (s)")
plt.ylabel(r"$\sigma$")
plt.title(r"Standard deviation of $g_2 (\tau)$")
plt.show()

We notice two features about this plot:
1. First, the step-like behavior, which stems from the stepped bin width used in the correlator.
2. Second, the non-decreasing trend: while the noise initially decreases with `tau`, it later increases again starting from about 1 ms.
This behavior is inconsistent with experimental observations, which show a decreasing trend over the whole time delay range.
This failure of the noise model at high delays has been previously observed in the scientific literature.

Depending on our application, we might want to modify the output of this function, for example by manually setting `sigma`
at the late bins to its minimum value to ensure a decreasing trend:

In [ ]:
idx_last_good = np.argmin(sigma)
sigma_corrected = sigma.copy()
sigma_corrected[idx_last_good + 1:] = sigma_corrected[idx_last_good]
plt.semilogx(tau, sigma, label="Original")
plt.semilogx(tau, sigma_corrected, linestyle="", marker=".", label="Corrected")
plt.xlabel(r"$\tau$ (s)")
plt.ylabel(r"$\sigma$")
plt.legend()
plt.show()

Now that we have the noise model, we can use it to add noise to our autocorrelation curves to mimic a real measurement. To do this,
once we have calculated the noiseless `g2_norm`, we sample from a Gaussian distribution with mean `g2_norm` and standard deviation
`sigma`.
Remember that the `sigma_g2_norm` function takes as input `tau_c`, the time constant of the exponential decay. To estimate this, the
noiseless `g2_norm` curve should be fitted with the exponential function defined above.

The `noise` module provides a `NoiseAdder` class that automates this process. The constructor take as input the integration time,
countrate, beta, and number of speckles, while the `add_noise` method takes `tau` and `g2_norm` as inputs and returns the noisy
`g2_norm_noisy`.

Let's see an example. For simplicity's sake, we'll use the semi-infinite geometry with Brownian motion again, but the noise addition
can be applied to any `g2_norm` curve.

In [ ]:
mua = 0.1
musp = 10
rho = 2.5
n = 1.4
db = 1e-8
beta = 0.5

msd = common.msd_brownian(tau, db)
g1_norm = hsi.g1_norm(msd, mua, musp, rho, n, lambda0)
g2_norm = 1 + beta * g1_norm**2

t_integration = 1   # Integration time, s
countrate = 70_000  # Measurement countrate, Hz
n_speckle = 1       # Number of speckles detected in parallel

noise_adder = noise.NoiseAdder(t_integration, countrate, beta, n_speckle)
g2_norm_noisy = noise_adder.add_noise(tau, g2_norm)

# Plotting
plt.semilogx(tau, g2_norm, label='Noiseless')
plt.semilogx(tau, g2_norm_noisy, marker='.', linestyle='', label='Noisy')
plt.xlabel(r'$\tau$ (s)')
plt.ylabel(r'$g_2$')
plt.legend()
plt.show()

Notice how the noise decreases with increasing `tau`, as expected. However, for large `tau`, it increases again due to
the limitations of the noise model discussed earlier. To mitigate this, the `NoiseAdder` class constructor also accepts
an optional boolean argument `ensure_decreasing` (default `False`): when set to `True`, it applies the correction we saw earlier
to ensure a decreasing noise trend.

In [ ]:
noise_adder = noise.NoiseAdder(t_integration, countrate, beta, n_speckle, ensure_decreasing=True)
g2_norm_noisy = noise_adder.add_noise(tau, g2_norm)

# Plotting
plt.semilogx(tau, g2_norm, label='Noiseless')
plt.semilogx(tau, g2_norm_noisy, marker='.', linestyle='', label='Noisy')
plt.xlabel(r'$\tau$ (s)')
plt.ylabel(r'$g_2$')
plt.legend()
plt.show()

As we can see, now the noise decreases with increasing `tau`, which is more realistic.

Now try simulating several noisy autocorrelation curves for different Brownian diffusion coefficients `db`.
Start by creating the noiseless curves `g2_all`, then use the `NoiseAdder` class to add noise to each of them.
Note that `NoiseAdder` works with 2D arrays as well, so you can pass the entire `g2_all` array at once and get
the noisy version `g2_all_noisy` directly back.

In [ ]:
db_vals = []  # Your code here...

# Noise parameters, feel free to change them
t_integration = 1
countrate = 70_000
n_speckle = 1

# Replace the zeros initialization with your code...
g1_all = np.zeros((len(db_vals), len(tau)))
g2_all = np.zeros((len(db_vals), len(tau)))
g2_all_noisy = np.zeros((len(db_vals), len(tau)))

# Plotting
for i in range(len(db_vals)):
    plt.semilogx(tau, g2_all[i, :], alpha=0.3, color=f"C{i}")
    plt.semilogx(tau, g2_all_noisy[i, :], '.', color=f"C{i}")
plt.xlabel(r'$\tau$ (s)')
plt.ylabel(r'$g_2$')
plt.show()

Finally, try changing the noise parameters (integration time, countrate, number of speckles) to see how they affect the noisy curves.

# Analyzing DCS data

In addition to forward modeling, the `Fit-DCS` toolbox also provides tools for solving the inverse problem, that is,
extracting scatterer dynamics from measured autocorrelation functions. These are implemented in the `inverse` module, which
comprises two submodules: `fit` and `mbl_homogeneous`.

We will start with the `fit` submodule, which contains the general-purpose `Fitter` class for fitting DCS autocorrelation data using
arbitrary forward models.

Let's start by creating some synthetic data to fit. We'll start by simulating `db` changes over time to mimic an occlusion experiment.

In [ ]:
baseline_value = 5e-9
baseline_length = 20
occlusion_value = 3e-10
occlusion_length = 20
peak_value = 6 * baseline_value
rising_time = 5
falling_time = 20
total_dur = baseline_length + occlusion_length + rising_time + falling_time
db = np.zeros(total_dur)
db[:baseline_length] = baseline_value
db[baseline_length:baseline_length + occlusion_length] = occlusion_value
db[baseline_length + occlusion_length:baseline_length + occlusion_length + rising_time] = np.linspace(occlusion_value, peak_value, rising_time)
db[baseline_length + occlusion_length + rising_time:baseline_length + occlusion_length + rising_time + falling_time] = np.linspace(peak_value, baseline_value, falling_time)

plt.plot(db)
plt.xlabel("Iteration")
plt.ylabel(r"$D_B$ (cm$^2$/s)")
plt.show()

Now use the semi-infinite forward model to simulate `g2_norm` curves for each `db` value, and add noise to them using the `NoiseAdder` class.
Remember that `g2_norm` should be a 2D array with shape `(total_dur, len(tau))`.

In [ ]:
mua = 0.1
musp = 10
rho = 2.5

# Replace the zeros initialization with your code...
g1_all = np.zeros((total_dur, len(tau)))
g2_all = np.zeros((total_dur, len(tau)))
g2_all_noisy = np.zeros((total_dur, len(tau)))

# Plotting a few curves
def preview_g2_curves(tau, g2_multi_noiseless, g2_multi_noisy, step=10):
    """
    Helper function to plot noiseless and noisy g2 curves every 'step' iterations.
    """
    n_curves = g2_all.shape[0]
    for i in range(0, n_curves, step):
        plt.semilogx(tau, g2_multi_noiseless[i, :], color=f"C{i//step}", label=i)
        plt.semilogx(tau, g2_multi_noisy[i, :], '.', color=f"C{i//step}")
    plt.xlabel(r'$\tau$ (s)')
    plt.ylabel(r'$g_2$')
    plt.legend(title="Iteration")
    plt.show()

preview_g2_curves(tau, g2_all, g2_all_noisy, step=10)

## Semi-infinite fitting with Brownian motion

The `inverse.fit.Fitter` class is used by instantiating it with the appropriate parameters and then calling its `fit()` method
to perform the fitting procedure on the data.

First of all, we need to choose the geometric model to use for fitting. This is done by passing the appropriate `g1_norm` function
from the `forward` module as an argument to the `Fitter` class constructor.
In our case, we will use the semi-infinite geometry, so we will pass `forward.homogeneous_semi_inf.g1_norm`.


In [ ]:
g1_fitting_function = hsi.g1_norm

Then, we need to define the scatterer motion model to use for fitting. This includes the model itself (e.g., Brownian, ballistic, hybrid),
the initial value of the parameter(s) to fit (e.g., `db` and/or `v_ms`), and their bounds for fitting.
To this end, the `inverse.fit` module provides the `MSDModelFit` class, allowing to define all these aspects in a convenient way.

Let's define a Brownian motion model for fitting, with an initial guess of `1e-8 cm^2/s` for `db` and lower bound `0` (no upper bound).

In [ ]:
import fit_dcs.inverse.fit as fit

msd_model = fit.MSDModelFit(model_name="brownian", params_init={"db": 1e-8}, params_bounds={"db": (0, None)})

Then, we need to define how to handle the coherence factor `beta` during fitting. This is done by instantiating the `BetaCalculator` class
from the `inverse.fit` module, which allows to choose between three different modes for estimating `beta`:
- `fixed`: use a fixed value for `beta` during fitting (passed as an argument to the constructor)
- `raw`: estimate `beta` from the raw `g2` data before fitting
- `fit`: treat `beta` as an additional fitting parameter

In our case, since we know the true value of `beta` used to generate the data, we will start simple and use the `fixed` strategy.

In [ ]:
beta_calculator = fit.BetaCalculator(mode="fixed", beta_fixed=beta)

To define the fitting range we can pass two more parameters: `tau_lims_fit`, a 2-tuple with the extreme values of `tau` used
for fitting (that is, `g2_norm` is fitted between `tau_lims_fit[0]` and `tau_lims_fit[1]`), and `g2_lim_fit`, the lowest value
of `g2_norm` up to which the fit is done (that is, the points of `g2_norm` after it has crossed `g2_lim_fit` are not used in the
fitting). Both `tau_lims_fit` and `g2_lim_fit` are optional parameters: one can pass both, just one, or neither. If both are
passed, the fitting range will satisfy both of them. If neither is passed, the whole `g2_norm` curve will be used for fitting.

In our case, we will pass both:

In [ ]:
tau_lims_fit = (1e-7, 1e-3)
g2_lim_fit = 1.13  # Corresponds to g1 = 0.5 at beta = 0.5

Lastly, we need to pass the other parameters required by the forward model, such as `mua`, `musp`, `rho`, etc. Since these
depend on the geometry, they are passed as keyword arguments to the `Fitter` class constructor, meaning that their names must be
specified and must match the names expected by the `g1_norm` function, but they can be passed in any order.
In our case, we have `mua`, `musp`, `rho`, `n`, and `lambda0`.

Now we have all the ingredients to instantiate the `Fitter` class.

In [ ]:
fitter = fit.Fitter(
    g1_fitting_function,
    msd_model,
    beta_calculator,
    tau_lims_fit=tau_lims_fit,
    g2_lim_fit=g2_lim_fit,
    mua=mua,
    musp=musp,
    rho=rho,
    n=n,
    lambda0=lambda0
)

To perform the fit, we call `fitter.fit()`, passing `tau` and the `g2_norm` to fit. Remember that `g2_norm` must be a 2D array
with shape `(n_iterations, len(tau))`.

The output of the `fit()` method is a dictionary containing the fitting results, including the fitted parameters for each iteration,
the estimated `beta` values, the fitted `g2_norm` curves, and more.

Additionally, we can specify a `plot_interval` to plot 1 every `plot_interval` curves to check the fitting.
If `plot_interval` is not passed, or if it is `0`, no plotting is done.

In [ ]:
fitted_data = fitter.fit(tau, g2_all_noisy, plot_interval=20)

`fitted_data` is a dictionary containing the fitting results. Its values are 1D arrays with length equal to the number of iterations
(i.e., the first dimension of `g2_norm`). Let's check its keys.

In [ ]:
print(fitted_data.keys())

As we can see, `fitted_data` includes the fitted parameters (`db` in our case), the estimated `beta` values, and the `chi2` and `r2` fitting
quality metrics.

Let's plot the fitted `db` values against the ground truth.

In [ ]:
plt.plot(db, label="Ground truth")
plt.plot(fitted_data["db"], '.', label="Fitted")
plt.xlabel("Iteration")
plt.ylabel(r"$D_B$ (cm$^2$/s)")
plt.legend()
plt.show()

Since we fixed `beta` to its ground truth value, the fit worked quite well.

Now try changing the `BetaCalculator` to use the `raw` mode instead of `fixed`, and see how it affects the fitting results.
See the documentation of the `BetaCalculator` class for more details on how to use the `raw` mode.
Remember to re-instantiate the `Fitter` class with the new `BetaCalculator`.

In [ ]:
# Replace the zeros initialization with your code...
fitted_data = {"db": np.zeros_like(db), "beta": np.zeros_like(db), "chi2": np.zeros_like(db), "r2": np.zeros_like(db)}

# Plotting
fig, axs = plt.subplots(2, 1, sharex=True)
axs[0].plot(db, label="Ground truth")
axs[0].plot(fitted_data["db"], '.', label="Fitted")
axs[0].set_ylabel(r"$D_B$ (cm$^2$/s)")
axs[0].legend()

axs[1].plot(np.full_like(db, beta), label="Ground truth")
axs[1].plot(fitted_data["beta"], '.', label="Estimated")
axs[1].set_xlabel("Iteration")
axs[1].set_ylabel(r"$\beta$")
axs[1].set_ylim(0.4, 0.6)
plt.show()

Since the `raw` mode estimates `beta` from the noisy data before fitting, the results are obviously noisier than in the previous case.
You can try changing the portion of the `g2_norm` curve used for estimating `beta` to see if you can improve the results.

Finally, try using the `fit` mode for the `BetaCalculator`, which treats `beta` as an additional fitting parameter. Again,
see the documentation of the `BetaCalculator` class for more details on how to use the `fit` mode.

In [ ]:
# Replace the zeros initialization with your code...
fitted_data = {"db": np.zeros_like(db), "beta": np.zeros_like(db), "chi2": np.zeros_like(db), "r2": np.zeros_like(db)}

# Plotting
fig, axs = plt.subplots(2, 1, sharex=True)
axs[0].plot(db, label="Ground truth")
axs[0].plot(fitted_data["db"], '.', label="Fitted")
axs[0].set_ylabel(r"$D_B$ (cm$^2$/s)")
axs[0].legend()

axs[1].plot(np.full_like(db, beta), label="Ground truth")
axs[1].plot(fitted_data["beta"], '.', label="Estimated")
axs[1].set_xlabel("Iteration")
axs[1].set_ylabel(r"$\beta$")
axs[1].set_ylim(0.4, 0.6)
plt.show()

You should see that in this case fitting `beta` along with `db` yields better results than the `raw` mode.

## Comparison of different geometric models

The `Fitter` class is flexible and allows using different geometric models for fitting by simply passing a different
`g1_norm` function from the `forward` module and adjusting the keyword arguments passed to the constructor accordingly.

Let's try to generate the same synthetic data as before, but this time using the slab geometry in transmittance
with thickness `d = 1 cm`.

In [ ]:
rho = 0  # Lateral source-detector separation, in transmittance it's usually 0
d = 1

# Replace the zeros initialization with your code...
g1_all = np.zeros((total_dur, len(tau)))
g2_all = np.zeros((total_dur, len(tau)))
g2_all_noisy = np.zeros((total_dur, len(tau)))

preview_g2_curves(tau, g2_all, g2_all_noisy)

Now try fitting this new dataset using two different geometric models:
- the slab geometry in transmittance
- the semi-infinite geometry with `rho = d` (to mimic transmittance)

In both cases, use the Brownian motion model. You are free to choose the `BetaCalculator` mode and fitting ranges as you prefer.

In [ ]:
# Replace the zeros initialization with your code...
fitted_data_slab = {"db": np.zeros_like(db), "beta": np.zeros_like(db), "chi2": np.zeros_like(db), "r2": np.zeros_like(db)}
fitted_data_semi_inf = {"db": np.zeros_like(db), "beta": np.zeros_like(db), "chi2": np.zeros_like(db), "r2": np.zeros_like(db)}

# Plotting
plt.plot(db, label="Ground truth")
plt.plot(fitted_data_slab["db"], '.', label="Fitted slab")
plt.plot(fitted_data_semi_inf["db"], '.', label="Fitted semi-inf")
plt.xlabel("Iteration")
plt.ylabel(r"$D_B$ (cm$^2$/s)")
plt.legend()
plt.show()

You should observe that the semi-infinite model, while capturing the general trend, underestimates the true `db` values compared
to the slab model. You can try changing the simulation parameters (e.g., optical properties, slab thickness...) to see how they
affect the fitting results, i.e. how much error is introduced by using the semi-infinite approximation.

## Multi-curve fitting

Many DCS devices feature multiple detection channels to improve the SNR of the measurement. When all of them are used at the same
source-detector separation, they can be averaged to obtain a single `g2_norm` curve with higher SNR.
However, when the detection channels are located at different source-detector separations, each of them measures a different
`g2_norm` curve, and simply averaging them would lead to incorrect results.

As such, the `Fitter` class can also handle multi-curve fitting,
that is, fitting multiple `g2_norm` curves simultaneously to extract a single set of scatterer dynamics parameters. Using this feature
is as easy as passing multiple `rho` values as a tuple and a 3D array for `g2_norm`, where:
- the first dimension corresponds to different iterations
- the second dimension corresponds to different source-detector separations
- the third dimension corresponds to delay times

Note that from an implementation standpoint, there is nothing special about `rho`: any other forward model parameter can be
passed as a tuple to indicate that multiple values are to be used, and the `Fitter` class will handle it correctly, though
the most common use case is varying `rho`.

Let's see an example. We will simulate `g2_norm` curves at two different source-detector separations, `rho = 1.5 cm` and `rho = 3.0 cm`,
using the semi-infinite geometry and Brownian motion with the same `db` changes as before.

*Side note: `NoiseAdder` expects 2D arrays as input, but we can simply reshape our 3D `g2_all` array to 2D
before passing it to the `add_noise()` method, and then reshape the output back to 3D. This is completely safe,
since the noise addition is done independently for each curve.*

In [ ]:
rho_vals = (1.5, 3.0)

# Noiseless g2_all
g1_all = np.zeros((total_dur, len(rho_vals), len(tau)))
for i_rho in range(len(rho_vals)):
    for i_iter in range(total_dur):
        msd = common.msd_brownian(tau, db[i_iter])
        g1_all[i_iter, i_rho, :] = hsi.g1_norm(msd, mua, musp, rho_vals[i_rho], n, lambda0)
g2_all = 1 + beta * g1_all**2

# Noisy g2_all_noisy
t_integration = 1
countrate = 70_000
n_speckle = 1
noise_adder = noise.NoiseAdder(t_integration, countrate, beta, n_speckle, ensure_decreasing=True)
g2_all_noisy = noise_adder.add_noise(tau, g2_all.reshape(-1, len(tau))).reshape(g2_all.shape)  # Reshape to 2D for noise addition and back to 3D

# Plotting a few curves
fig, axs = plt.subplots(2, 1, sharex=True)
for i_rho in range(len(rho_vals)):
    ax = axs[i_rho]
    ax.set_title(f"rho = {rho_vals[i_rho]} cm")
    ax.set_ylabel(r"$g_2$")
    for i in range(0, total_dur, 10):
        ax.semilogx(tau, g2_all[i, i_rho, :], color=f"C{i//10}", alpha=0.3)
        ax.semilogx(tau, g2_all_noisy[i, i_rho, :], '.', color=f"C{i//10}")
axs[1].set_xlabel(r'$\tau$ (s)')
plt.show()

Now let's fit this multi-curve dataset using the semi-infinite geometry and Brownian motion. We simply need to pass
`rho_vals` as the `rho` argument to the `Fitter` class constructor, and the 3D array `g2_all_noisy` to the `fit()` method.

In [ ]:
beta_calculator = fit.BetaCalculator(mode="raw", tau_lims=(1e-7, 3e-7))
fitter = fit.Fitter(
    hsi.g1_norm,
    msd_model,
    beta_calculator,
    tau_lims_fit,
    g2_lim_fit,
    mua=mua,
    musp=musp,
    rho=rho_vals,
    n=n,
    lambda0=lambda0
)
fitted_data = fitter.fit(tau, g2_all_noisy, plot_interval=30)

If `plot_interval` is specified, the fitting plots will show all the curves for each iteration, automatically creating the necessary
subplots. The first subplot corresponds to the first `rho` value, the second subplot to the second `rho` value, and so on.

The output `fitted_data` dictionary is mostly the same as in the single-curve fitting case, with the only difference that now
multiple `beta` values are estimated (one for each `rho` value), and thus separate keys are created in the dictionary: `beta_0`, `beta_1`, etc.

In [ ]:
print(fitted_data.keys())

We can now plot the fitted `db` values against the ground truth as before.

In [ ]:
plt.plot(db, label="Ground truth")
plt.plot(fitted_data["db"], '.', label="Fitted")
plt.xlabel("Iteration")
plt.ylabel(r"$D_B$ (cm$^2$/s)")
plt.legend()
plt.show()

Now try it yourself! Create a multi-curve dataset using different `rho` values, and fit it using the `Fitter` class.
You are free to choose the simulation and fitting parameters, such as the geometric model, number of `rho` values, etc.

In [ ]:
rho_vals = ()  # Your code here...

# Replace the zeros initialization with your code...
fitted_data = {"db": np.zeros_like(db), "chi2": np.zeros_like(db), "r2": np.zeros_like(db)}

# Plotting
plt.plot(db, label="Ground truth")
plt.plot(fitted_data["db"], '.', label="Fitted")
plt.xlabel("Iteration")
plt.ylabel(r"$D_B$ (cm$^2$/s)")
plt.legend()
plt.show()

## Two-layer model

Multi-curve fitting is especially useful when dealing with layered geometries, where the sensitivity to different layers
varies with source-detector separation. The `Fitter` class works seamlessly with geometries that feature multiple scatterer
motion models, such as the two-layer geometry. We just need to define two `MSDModelFit` instances, one for each layer, and pass them
as a tuple to the `Fitter` class constructor.

Let's see an example. We will simulate `g2_norm` curves at two different source-detector separations, `rho = (1, 3) cm`,
using the bilayer geometry with Brownian motion in both layers. The upper layer will have thickness `d = 0.5 cm` and constant
`db_up = 1e-8 cm^2/s`, while the lower layer will have `db_dn` changing over time as before.

In [ ]:
db_up = 1e-8
d = 0.5
rho_vals = (1, 3)
mua_up = 0.1
mua_dn = 0.2
musp_up = 8
musp_dn = 10
q_max = 60

msd = np.zeros((2, len(tau)))
msd[0, :] = common.msd_brownian(tau, db_up)  # Upper layer MSD
for i_rho in range(len(rho_vals)):
    for i_iter in range(total_dur):
        msd[1, :] = common.msd_brownian(tau, db[i_iter])  # Lower layer MSD
        g1_all[i_iter, i_rho, :] = bilayer.g1_norm(msd, mua_up, mua_dn, musp_up, musp_dn, n, d, rho_vals[i_rho], lambda0, q_max)
g2_all = 1 + beta * g1_all**2

noise_adder = noise.NoiseAdder(t_integration, countrate, beta, n_speckle, ensure_decreasing=True)
g2_all_noisy = noise_adder.add_noise(tau, g2_all.reshape(-1, len(tau))).reshape(g2_all.shape)

# Plotting a few curves
fig, axs = plt.subplots(2, 1, sharex=True)
for i_rho in range(len(rho_vals)):
    ax = axs[i_rho]
    ax.set_title(f"rho = {rho_vals[i_rho]} cm")
    ax.set_ylabel(r"$g_2$")
    for i in range(0, total_dur, 10):
        ax.semilogx(tau, g2_all[i, i_rho, :], color=f"C{i//10}", alpha=0.3)
        ax.semilogx(tau, g2_all_noisy[i, i_rho, :], '.', color=f"C{i//10}")
axs[1].set_xlabel(r'$\tau$ (s)')
plt.show()

Now let's fit this multi-curve dataset using the bilayer geometry with Brownian motion in both layers.
We need to define two `MSDModelFit` instances, one for each layer, and pass them as a tuple to the `Fitter` class constructor.
To only fit `db_dn`, we will fix `db_up` to its ground truth value by setting both its initial value and bounds accordingly.

Additionally, since the bilayer model is more computationally intensive, we will speed up the fitting process by using multiple workers.
To do so, we pass the `num_workers` argument to the `fit()` method, specifying the number of parallel workers to use.

In [ ]:
msd_model_up = fit.MSDModelFit(model_name="brownian", params_init={"db": 1e-8}, params_bounds={"db": (1e-8, 1e-8)})
msd_model_dn = fit.MSDModelFit(model_name="brownian", params_init={"db": 1e-8}, params_bounds={"db": (0, None)})
msd_models = (msd_model_up, msd_model_dn)

beta_calculator = fit.BetaCalculator(mode="fixed", beta_fixed=beta)
fitter = fit.Fitter(
    bilayer.g1_norm,
    msd_models,
    beta_calculator,
    tau_lims_fit,
    g2_lim_fit,
    mua_up=mua_up,
    mua_dn=mua_dn,
    musp_up=musp_up,
    musp_dn=musp_dn,
    d=d,
    n=n,
    rho=rho_vals,
    lambda0=lambda0,
    q_max=q_max
)
fitted_data = fitter.fit(tau, g2_all_noisy, num_workers=15)

# Plotting
fig, axs = plt.subplots(2, 1, sharex=True)
axs[0].plot(db, label="Ground truth")
axs[0].plot(fitted_data["db_1"], '.', label="Fitted")
axs[0].set_ylabel(r"$D_B$ lower layer (cm$^2$/s)")
axs[0].legend()
axs[1].plot(np.full_like(db, db_up), label="Ground truth")
axs[1].plot(fitted_data["db_0"], '.', label="Fitted")
axs[1].set_xlabel("Iteration")
axs[1].set_ylabel(r"$D_B$ upper layer (cm$^2$/s)")
axs[1].legend()
plt.show()

In this case, since we fixed both `db_up` and `beta`, the fitting works quite well.

Now try changing some of the parameters to see how they affect the fitting results, such as:
- letting `db_up` vary during fitting
- changing the `BetaCalculator` mode
- changing the `rho` values used
- changing the upper layer thickness `d`
- ...

## Modified Beer-Lambert law

Data analysis via the Modified Beer-Lambert law is also supported. The necessary functionalities are implemented in the
`inverse.mbl_homogeneous` module. Similarly to `inverse.fit`, it provides a `MBLHomogeneous` class to handle the
analysis of the measurement.

Remember that the Modified Beer-Lambert law relates *changes* in autocorrelation to *changes* in `db` (or `v_ms`). That is to say,
to get absolute values of the parameters of interest, we first need to define a baseline from which changes occur. In order to do so,
we need `g2_norm_0`, the autocorrelation in baseline conditions, as well the baseline values of all the parameters: `mua0`, `musp0`,
and even `db0`.

Currently, only homogeneous geometries and scatterer motion models with a single parameter (i.e., Brownian or ballistic) are supported.

Let's see an example. We will create a synthetic dataset similar to the previous ones, with the semi-infinite geometry and Brownian motion.

In [ ]:
rho = 2.5
g1_all = np.zeros((total_dur, len(tau)))
for i in range(total_dur):
    msd = common.msd_brownian(tau, db[i])
    g1_all[i, :] = hsi.g1_norm(msd, mua, musp, rho, n, lambda0)
g2_all = 1 + beta * g1_all**2

t_integration = 1
countrate = 70_000
n_speckle = 1
noise_adder = noise.NoiseAdder(t_integration, countrate, beta, n_speckle, ensure_decreasing=True)
g2_all_noisy = noise_adder.add_noise(tau, g2_all)

# Plotting a few curves
for i in range(0, total_dur, 10):
    plt.semilogx(tau, g2_all[i, :], color=f"C{i//10}", label=i)
    plt.semilogx(tau, g2_all_noisy[i, :], '.', color=f"C{i//10}")
plt.xlabel(r'$\tau$ (s)')
plt.ylabel(r'$g_2$')
plt.legend(title="Iteration")
plt.show()

Now we need `g2_norm_0`, the baseline autocorrelation curve. While we could simply take the first curve of the noiseless dataset,
to be more realistic we will average the first `baseline_length` curves of the noisy dataset to obtain a better estimate of `g2_norm_0`.

In [ ]:
g2_norm_0 = np.mean(g2_all_noisy[:baseline_length, :], axis=0)
plt.semilogx(tau, g2_norm_0)
plt.xlabel(r'$\tau$ (s)')
plt.ylabel(r'$g_2$ baseline')
plt.show()

In a real setting `mua0` and `musp0` would come from a TD NIRS measurement, and we could average them over the baseline as we did for `g2_norm_0`.
In our case we modeled constant optical properties, so `mua0 = mua` and `musp0 = musp`.

As for `db0`, for simplicity we will set it to the ground truth baseline value used to generate the data, that is, `db0 = baseline_value`.
In a real setting, `db0` could be estimated via fitting of the baseline `g2_norm_0` curve using the `Fitter` class we saw earlier
(which you will be doing in the next exercise).

In [ ]:
db0 = baseline_value

Now we need a model for scatterer motion, and we can specify it as an instance of the `inverse.mbl_homogeneous.MSDModelMBL` class,
similarly to what we did earlier with `inverse.fit.MSDModelFit`. Along with the model, we need to specify the baseline value
of the parameter (`db0` in our case).

In [ ]:
import fit_dcs.inverse.mbl_homogeneous as mbl

msd_model = mbl.MSDModelMBL(model_name="brownian", param0=db0)

Lastly, we need to specify the geometric model. Similarly to `Fitter`, we do so by passing the appropriate function to the
`MBLHomogeneous` class constructor. In this case, however, the function to be passed is not the fitting `g1_norm`, but rather a function
that returns the sensitivity coefficients for the geometry of interest, which can be found in the `forward` module
(see their documentation for details). As a side note, it is also possible to pass a custom function for computing sensitivity coefficients.
This is useful when calculating them through other means, e.g., Monte Carlo simulations.

In our case, we are going to use `forward.homogeneous_semi_inf.d_factors`, which computes the sensitivity coefficients for the
homogeneous semi-infinite geometry.

In [ ]:
d_factors_fn = hsi.d_factors

This function, much like `g1_norm`, requires several parameters to compute the sensitivity coefficients, such as `mua`, `musp`, `rho`, etc.
As was the case for `Fitter`, these are passed as keyword arguments to the `MBLHomogeneous` class constructor.

We are now ready to analyze the data using the Modified Beer-Lambert law. Let's create our `mbl_analyzer` with all the options we defined.
Note the difference between `mua` and `mua0` (similarly for `musp` and `musp0`): the former are the current optical properties
used for analysis, while the latter are the baseline optical properties used for computing sensitivity coefficients.
In this case they are the same, but in a real setting they could differ to account for changes in optical properties during the measurement.
Then, `mua` and `musp` would be arrays with length equal to the number of iterations, while `mua0` and `musp0` would be scalars.
Note that passing arrays for `mua` and `musp` is supported also by the `Fitter` class, though for simplicity we didn't use this
feature in the previous examples.

In [ ]:
mbl_analyzer = mbl.MBLHomogeneous(
    g2_norm_0,
    d_factors_fn,
    msd_model,
    mua,
    musp,
    mua0=mua,
    musp0=musp,
    rho=rho,
    n=n,
    lambda0=lambda0
)

We can call `mbl_analyzer.fit(tau, g2_all_noisy)` to perform the analysis. Note that plotting during the analysis is not supported, since there is
no fitting function to plot. Also, multi-threading is not supported, since the analysis is already very fast.

We will typically get warnings saying that we tried to divide by 0, or that we tried to take the logarithm of a negative number. This
happens because of the noise, and is completely normal: `g2_norm` at later delay times has a low SNR and results in invalid values of `db`.

In [ ]:
db_mbl = mbl_analyzer.fit(tau, g2_all_noisy)

The `fit()` method returns a matrix the same shape as `g2_norm`, containing the value of the parameter of interest (`db`, in our case)
for each iteration and lag time. We can plot the mean and standard deviation of `db_mbl` during the baseline to assess which lag
times to consider:

In [ ]:
db_mbl_baseline_avg = np.mean(db_mbl[:baseline_length, :], axis=0)
db_mbl_baseline_sd = np.std(db_mbl[:baseline_length, :], axis=0)
plt.hlines(baseline_value, tau[0], 4e-4, linestyle="--", color="tab:blue", label="Ground truth")
plt.errorbar(tau, db_mbl_baseline_avg, db_mbl_baseline_sd, marker="o", color="tab:orange", label="MBL baseline")
plt.xscale("log")
plt.xlabel(r"$\tau$")
plt.ylabel(r"D$_B$ (cm$^2$/s)")
plt.legend()
plt.ylim((0, 2 * baseline_value))
plt.tight_layout()
plt.show()

The signal is very noisy at small lag times, but for big values of `tau` the Modified Beer-Lambert law performs poorly
(notice the missing points: our `tau` actually extends to `1 s`!). A good trade-off in this case could be `1e-5 s < tau < 3e-5 s`.

In [ ]:
mask = np.logical_and(tau > 1e-5, tau < 3e-5)
db_mbl_signal = np.mean(db_mbl[:, mask], axis=1)
plt.plot(db, linestyle="--", label="Ground truth")
plt.plot(db_mbl_signal, label="MBL")
plt.legend()
plt.xlabel("Iteration")
plt.ylabel(r"D$_B$ (cm$^2$/s)")
plt.show()

Now try changing the `tau` interval used for analysis to see how it affects the results. You should observe that
earlier delay times give noisier signals, while later delays result in cleaner signals, but tend to underestimate big changes such
as those occurring in the hyperemic peak.

In [ ]:
# Define 3 masks for different tau intervals
# Replace the following line with your code...
mask1, mask2, mask3 = mask, mask, mask

# Apply masks
db1 = np.mean(db_mbl[:, mask1], axis=1)
db2 = np.mean(db_mbl[:, mask2], axis=1)
db3 = np.mean(db_mbl[:, mask3], axis=1)

# Plotting
plt.plot(db, linestyle="--", label="Ground truth")
plt.plot(db1, label="Tau interval 1")
plt.plot(db2, label="Tau interval 2")
plt.plot(db3, label="Tau interval 3")
plt.legend()
plt.xlabel("Iteration")
plt.ylabel(r"D$_B$ (cm$^2$/s)")
plt.show()

Now try comparing the results of the Modified Beer-Lambert law with those obtained via fitting using the `Fitter` class.
Generate a new synthetic dataset using the slab geometry in transmittance, and analyze it using both methods.
You are free to choose the fitting parameters and `tau` intervals as you prefer. For the MBL analysis, use the fitting results
as baseline values for `db0`.

In [ ]:
# Simulation parameters
# Your code here...

# Noiseless dataset
# Your code here...

# Noisy dataset
# Your code here...

# Fitting with Fitter
# Replace the zeros initialization with your code...
fitted_data = dict(
    db=np.zeros_like(db),
    beta=np.zeros_like(db),
    chi2=np.zeros_like(db),
    r2=np.zeros_like(db)
)
db_fit = fitted_data["db"]

# Analysis with MBL
# Replace the zeros initialization with your code...
db_mbl = np.zeros_like(db)


# Plotting
plt.plot(db, linestyle="--", label="Ground truth")
plt.plot(db_fit, label="Fitted")
plt.plot(db_mbl, label="MBL")
plt.legend()
plt.xlabel("Iteration")
plt.ylabel(r"D$_B$ (cm$^2$/s)")
plt.show()

## Saving and loading analysis results

While there is no built-in functionality to save the analysis results to disk, it is straightforward to do so using
standard Python libraries such as `numpy` or `pandas`.

`numpy` provides the `np.savez()` function to save multiple arrays to a single binary file in `.npz` format, and the `np.load()`
function to load them back. Here's an example of how to save the fitting results obtained with the `Fitter` class.

In [ ]:
output_filename = "fitting_results.npz"
np.savez(output_filename, db=fitted_data["db"], beta=fitted_data["beta"])

And to load them back:

In [ ]:
loaded_data = np.load(output_filename)
db_loaded, beta_loaded, = loaded_data["db"], loaded_data["beta"]
plt.plot(db_loaded, label="Loaded db")
plt.xlabel("Iteration")
plt.ylabel(r"$D_B$ (cm$^2$/s)")
plt.legend()
plt.show()

Alternatively, `pandas` provides the `DataFrame.to_csv()` method to save tabular data to a CSV file, and the `pd.read_csv()`
function to load it back. This is especially useful if we want to be able to easily inspect and manipulate the results
using spreadsheet software.

Here's an example of how to save the fitting results using `pandas`. First, we convert the `fitted_data` dictionary to a `DataFrame`,
then we save it to a CSV file.

In [ ]:
import pandas as pd

results_df = pd.DataFrame(fitted_data)
results_df.to_csv("fitting_results.csv", index=False)

To load the results back:

In [ ]:
loaded_results_df = pd.read_csv("fitting_results.csv")
loaded_results_df

# Loading real measurement data

`Fit-DCS` also provides tools to load real measurement data from different DCS devices. The relevant classes are implemented
in the `utils.data_loaders` module, which can read data files from two types of devices: the ALV7004/USB-FAST hardware correlator,
and the Swabian TimeTagger20.

## Hardware correlator

Let's start with the hardware correlator. The `ALVDataLoader` class can read `.asc` files generated by the ALV software (each file
corresponds to an iteration). To use it, first we create an instance by passing the list of data files to the class constructor,
and then we call the `load_data()` method on the instance.
Note that `load_data()` does not return anything, rather, it stores the info about the measurement in the class attributes.

We can then call `loader.get_data()` to retrieve the data as a dictionary (see below for details on its contents).

Let's see an example. We will load the measurement contained in the `data/PRO_025_T2_Occ` folder, which contains a DCS measurement
of a *vastus lateralis* muscle during a cuff occlusion protocol. The `os.listdir()` function is used to get the list of data files
in the folder.

In [ ]:
import os
import fit_dcs.utils.data_loaders as data_loaders

data_files = os.listdir("data/hardware_corr/")[:-1]   # We exclude the last file, since it's incomplete and can't be read properly
data_files = [os.path.join("data/hardware_corr/", file) for file in data_files]
loader = data_loaders.DataLoaderALV(data_files)
loader.load_data()
alv_data = loader.get_data()
print(alv_data.keys())

As we can see, `data` is a dictionary containing the measurement data, including:
- `tau`: 1D array of time delays
- `g2_norm`: 3D array of normalized g2 data, with shape `(n_iterations, n_channels, n_bins)`
- `countrate`: 2D array of countrate values, with shape `(n_iterations, n_channels)`
- `integration_time`: scalar value storing the integration time used for the measurement, in seconds

`g2_norm` is the shape requested by the `Fitter` class for multi-curve fitting, streamlining the analysis process.
Of course, we can also average over the channels to obtain a 2D array if we want to perform single-curve fitting or a
Modified Beer-Lambert law analysis. To this end, `utils.data_loaders` also provides a helper function, `weigh_g2()`,
which performs a weighted average of `g2_norm` using the countrate values as weights.

Here's how to use it:

In [ ]:
g2_avg = data_loaders.weigh_g2(alv_data["g2_norm"], alv_data["countrate"])

# Plotting the curves
plt.semilogx(alv_data["tau"], g2_avg.T)
plt.xlabel(r'$\tau$ (s)')
plt.ylabel(r'$g_2$ averaged')
plt.show()

As we can see, the measured `g2_norm` curves exhibit the afterpulse effect at short delay times, which should be excluded
from the analysis. We can either apply a mask to the curves before fitting, or use the `tau_lims_fit` argument of the `Fitter` class
to exclude the first delay times during fitting.

Now try fitting this dataset using the `Fitter` class. The measurement was taken at `rho = 2.5 cm`, and we can assume
typical optical properties for muscle tissue at NIR wavelengths: `mua = 0.2 cm^-1`, `musp = 10 cm^-1`. You are free to choose
the scatterer motion model, `BetaCalculator` mode, and fitting parameters as you prefer.

In [ ]:
# Replace the zeros initialization with your code...
db_fitted = np.zeros(alv_data["g2_norm"].shape[0])

# Plotting
plt.plot(db_fitted)
plt.xlabel("Iteration")
plt.ylabel(r"$D_B$ (cm$^2$/s)")
plt.show()

## TimeTagger

Now let's see how to load data from the Swabian TimeTagger20.
The `.ttbin` files saved by the TimeTagger require the Swabian libraries to be read, which can be freely downloaded from
https://www.swabianinstruments.com/time-tagger/downloads/. Once those are installed (only the Python files are needed), the data loader
will function properly.

The `.ttbin` files store the raw time tags, from which we compute the autocorrelations. The first step is to define the architecture of
the software correlator. Since we are using a multi-tau correlator, the architecture can be specified through the following 3 integers:
* `s`: the number of cascaded linear correlators.
* `m`: the binning ratio between two consecutive linear correlators (in hardware correlators usually `m = 2`).
* `p`: the number of bins in each linear correlator, it must be an integer multiple of `m`

While one can manually set `s`, `m`, and `p`, the `utils.timetagger` module provides the `get_correlator_architecture` function, which takes
as input the following arguments:
* `alpha = p / m`, which should be high enough to guarantee a good signal-to-noise ratio. For example, `alpha = 7` guarantees an
error the order of `1e-3`.
* `m`.
* `tau_max`, the maximum desired value of `tau` up to which to calculate the autocorrelation.
* `t0`, the resolution (in s) of the TimeTagger. The Swabian TimeTagger20 has a resolution of 1 ps, so we should set `t0 = 1e-12`.

`get_correlator_architecture` will then output `p` and `s`, like so:

In [ ]:
from fit_dcs.utils.timetagger import get_correlator_architecture

m = 2
(p, s) = get_correlator_architecture(alpha=7, m=m, tau_max=1e-2, t0=1e-12)
print(f"m={m}, p={p}, s={s}")

Now that we have defined the correlator architecture, we can load the data using the `data_loaders.DataLoaderTimeTagger` class, which
reads a measurement file and calculates the autocorrelation according to the specified architecture and integration time.

Let's load the measurement `2024-05-07_Pulsatility_forearm.ttbin`, contained in `data/software_corr`. Measurement files generated by the
TimeTagger consist of a header file (in our case `2024-05-07_Pulsatility_forearm.ttbin`), with information on the measurement, and one or more
files which actually store the time tags (in our case there are `3`). We only pass the header file to the data loader, which automatically
takes care of loading the time tags from the other files.

This measurement features a high countrate (> 1 MHz) on channel 1, and only lasts a few seconds, so we are going to use a short integration
time, `30 ms`, to calculate the autocorrelations. To speed up the computation, we can pass the optional argument `tau_start`, which is the
minimum value of `tau` from which the autocorrelation is calculated. Since the detectors we use have a `20 ns` dead time, the autocorrelation
for `tau < 20 ns` will always be `0`, so we should set `tau_start` to at least `20e-9`. Since early lag times feature the afterpulse peak,
we will set it a little higher, to `100 ns`.

Note that the `channels` argument must be a list even when using only 1 channel.

In [ ]:
file = "data/software_corr/2024-05-07_Pulsatility_forearm.ttbin"
integration_time = 30e-3
channels = [1]
tau_start = 1e-7
loader = data_loaders.DataLoaderTimeTagger(
    file,
    integration_time=integration_time,
    channels=channels,
    p=p,
    m=m,
    s=s,
    tau_start=tau_start
)

Similar to `DataLoaderALV`, we call `loader.load_data()`, which stores the results in the attributes `tau`, `g2_norm`, and `countrate`.
`load_data()` calculates the autocorrelations on each channel in parallel, and we can set the maximum number of parallel workers
via the `max_workers` argument. If we don't set it, it will default to using all available CPU cores.

In this case we are only analyzing 1 channel, so there is no need to use more than 1 worker. Note that if we set a
higher number of workers than channels, `load_data()` will simply use the number of channels as `max_workers`.

In [ ]:
loader.load_data(max_workers=1)

Just like with `DataLoaderALV`, we can retrieve the data via `loader.get_data()` to get a dictionary with keys `tau`, `g2_norm`,
`countrate` and `integration_time`.
Additionally, `DataLoaderTimeTagger.get_data()` also accepts a boolean `full` argument (default `True`), which indicates whether to
return `tau` and `g2_norm` including all lag times (if `full=True`), or only those for `tau >= tau_start` (if `full=False`).
In this case, since we set `tau_start` to `100 ns`, we will set `full=False` to retrieve only the relevant part of the autocorrelation.

In [ ]:
pulsatility_data = loader.get_data(full=False)
tau, g2_norm, countrate = pulsatility_data["tau"], pulsatility_data["g2_norm"], pulsatility_data["countrate"]
print(g2_norm.shape)

Note that, even though we only analyzed 1 channel, `g2_norm` is still a 3D array with shape `(n_iterations, n_channels, n_bins)`.
This is not a problem for the `Fitter` class, which can handle both single-curve and multi-curve fitting seamlessly. However, depending
on your downstream analysis, you might want to squeeze the array to remove the singleton dimension corresponding to channels.

Now try fitting this dataset using the `Fitter` class. The measurement was taken at `rho = 1 cm`, and we can assume
typical optical properties for forearm tissue at NIR wavelengths: `mua = 0.2 cm^-1`, `musp = 10 cm^-1`. You are free to choose
the scatterer motion model, `BetaCalculator` mode, and fitting parameters as you prefer.

In [ ]:
# Replace the zeros initialization with your code...
db_fitted = np.zeros(g2_norm.shape[0])

# Plotting
plt.plot(db_fitted)
plt.xlabel("Iteration")
plt.ylabel(r"$D_B$ (cm$^2$/s)")
plt.show()